In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [ ]:
def _baseMeteorologia(db_conn):   

    data_Meteo = _consulta_meteoUS_full(db_conn)

    # Se agrega campo FECHA para join
    data_Meteo['FECHA'] =pd.to_datetime({'year': data_Meteo['ANIO'],
                                              'month': data_Meteo["MES"],
                                              'day':  data_Meteo['DIA']},
                                              format='%d-%m-%Y', errors='coerce')

    data_Meteo['TEMPERATURA'] = pd.to_numeric(data_Meteo['TEMPERATURA'])
    data_Meteo['VIENTO'] = pd.to_numeric(data_Meteo['VIENTO'])
    data_Meteo['PRESION'] = pd.to_numeric(data_Meteo['PRESION'])
    data_Meteo['HUMEDAD'] = pd.to_numeric(data_Meteo['HUMEDAD'])
    data_Meteo['PRECIPITACION_1h'] = pd.to_numeric(data_Meteo['PRECIPITACION_1h'])
    data_Meteo['PRECIPITACION_3h'] = pd.to_numeric(data_Meteo['PRECIPITACION_3h'])

    data_Meteo = data_Meteo.groupby(['ANIO', 'MES', 'DIA', 'FECHA']).agg(TEMP_MAX= ('TEMPERATURA','max'),
                                     TEMP_MIN = ('TEMPERATURA','min'),
                                     HUMEDAD = ('HUMEDAD','mean'),
                                     VIENTO = ('VIENTO','mean'),
                                     PRESION = ('PRESION', 'mean'),
                                     PRECIPITACION_1h = ('PRECIPITACION_1h', 'mean'),
                                     PRECIPITACION_3h = ('PRECIPITACION_3h', 'mean'),
                                     DESC_TIEMPO = ('DESC_TIEMPO', lambda x: x.value_counts().index[0]))

    data_Meteo.reset_index(inplace=True)
    
    return data_Meteo

In [ ]:
# FECHAS DE FESTIVOS
def _baseFestivos():
    df_festivos = pd.read_csv('../../data/Festivos_Madrid.csv', sep=';')
    # Se agrega campo FECHA para join
    df_festivos['FECHA'] =pd.to_datetime({'year': df_festivos['ANIO'],
                                              'month': df_festivos["MES"]
                                              ,'day':  df_festivos['DIA']},
                                              format='%d-%m-%Y', errors='coerce')
    return df_festivos

In [ ]:
def _agregaRegistrosParaDummies(listaTemporada, listaDescTiempo):
    
    df_RegistroDummy = pd.DataFrame(columns=['ESTACION',
                                             'ANIO',
                                             'MES',
                                             'DIA',
                                             'TEMPORADA',
                                             'DIA_SEMANA',
                                             'Es_Festivo',
                                             'Es_FinSemana',
                                             'TEMP_MAX',
                                             'TEMP_MIN',
                                             'VIENTO',
                                             'PRESION',
                                             'HUMEDAD',
                                             'PRECIPITACION_1h',
                                             'PRECIPITACION_3h',
                                             'DESC_TIEMPO'])
    
    for i in range(0,12): #Meses
        for j in range(listaTemporada.size): #TEMPORADA
            for k in range(0,7): # DIA_SEMANA
                for l in range(listaDescTiempo.size): #DESC_TIEMPO
                    for m in range(0,2): #Es_Festivo
                        for n in range(0,2): #Es_FinSemana
                            nrow = {
                                'ESTACION':0,
                                'ANIO':0,
                                'MES':i+1,
                                'DIA':0,
                                'TEMPORADA':listaTemporada[j],
                                'DIA_SEMANA':k+1,
                                'Es_Festivo':m,
                                'Es_FinSemana':n,
                                'TEMP_MAX':0,
                                'TEMP_MIN':0,
                                'VIENTO':0,
                                'PRESION':0,
                                'HUMEDAD':0,
                                'PRECIPITACION_1h':0,
                                'PRECIPITACION_3h':0,
                                'DESC_TIEMPO':listaDescTiempo[l]
                            }
                            df_RegistroDummy = pd.concat([df_RegistroDummy, pd.DataFrame(nrow, index=[0])])
        #.append(df_row, ignore_index=True)
        
                
    return df_RegistroDummy

In [ ]:
def _estacionDataFechaNueva(idEstacion, fecha, data_Meteo_in, df_festivos_in, df_Dummies, lst_CatCol, lst_NumCol):
    
    # Variable dia semana
    if fecha.isoweekday()==7:
        dia_semana=1
    else:
        dia_semana=fecha.isoweekday()+1

    # Variable Es_FinSemana
    if ((dia_semana==1) or (dia_semana==7)):
        es_finsemana = 1
    else:
        es_finsemana = 0

    # Variable TEMPORADA
    if fecha.month<=3:
        temporada='INVIERNO'
    elif fecha.month<=6:
        temporada='PRIMAVERA'
    elif fecha.month<=9:
        temporada='VERANO'
    else:
        temporada='OTONO'
    
    meteo = data_Meteo_in[data_Meteo_in['FECHA']==fecha.strftime('%m/%d/%Y')]
    es_festivo = df_festivos_in[df_festivos_in['FECHA']==fecha.strftime('%m/%d/%Y')]['FESTIVO'].iloc[0]
    
    new_row_dict = {
        'ESTACION': idEstacion,
        'ANIO': fecha.year,
        'MES': fecha.month,
        'DIA': fecha.day,
        'TEMPORADA': temporada,
        'DIA_SEMANA': dia_semana,
        'Es_Festivo': es_festivo,
        'Es_FinSemana': es_finsemana,
        'TEMP_MAX': float(meteo.TEMP_MAX.iloc[0]),
        'TEMP_MIN': float(meteo.TEMP_MIN.iloc[0]),
        'HUMEDAD': float(meteo.HUMEDAD.iloc[0]),
        'VIENTO': float(meteo.VIENTO.iloc[0]),
        'PRESION': float(meteo.PRESION.iloc[0]),
        'PRECIPITACION_1h': float(meteo.PRECIPITACION_1h.iloc[0]),
        'PRECIPITACION_3h': float(meteo.PRECIPITACION_3h.iloc[0]),
        'DESC_TIEMPO': meteo.DESC_TIEMPO.iloc[0]
    }

    fecha = pd.DataFrame(new_row_dict, index=[0])
    
    fecha['MES'] = fecha['MES'].astype('category')
    fecha['DIA_SEMANA'] = fecha['DIA_SEMANA'].astype('category')
    fecha['TEMPORADA'] = fecha['TEMPORADA'].astype('category')
    fecha['Es_Festivo'] = fecha['Es_Festivo'].astype('category')
    fecha['Es_FinSemana'] = fecha['Es_FinSemana'].astype('category')
    fecha['DESC_TIEMPO'] = fecha['DESC_TIEMPO'].astype('category')


    # Se agrega DF con valores para dummies
    fecha = pd.concat([fecha, df_Dummies])
    
    # Se generan campos dummies
    fecha = pd.get_dummies(fecha, columns=lst_CatCol, drop_first=True)
        
    X = fecha[['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION','PRECIPITACION_1h','PRECIPITACION_3h']]
    
    scaler = StandardScaler()
    fecha[['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION','PRECIPITACION_1h','PRECIPITACION_3h']] = scaler.fit_transform(X)
    
    # Se eliminan registros usados para generar variables dummies
    fecha = fecha.iloc[0]
    
    fecha = fecha[['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION',
                  'PRECIPITACION_1h','PRECIPITACION_3h','MES_2','MES_3','MES_4','MES_5',
                  'MES_6','MES_7','MES_8','MES_9','MES_10','MES_11','MES_12','DIA_SEMANA_2',
                  'DIA_SEMANA_3','DIA_SEMANA_4','DIA_SEMANA_5','DIA_SEMANA_6','DIA_SEMANA_7',
                  'TEMPORADA_OTONO','TEMPORADA_PRIMAVERA','TEMPORADA_VERANO','Es_Festivo_1','Es_FinSemana_1',
                  'DESC_TIEMPO_Clouds','DESC_TIEMPO_Drizzle','DESC_TIEMPO_Fog','DESC_TIEMPO_Mist',
                  'DESC_TIEMPO_Rain','DESC_TIEMPO_Snow','DESC_TIEMPO_Thunderstorm']]
        
    fecha = fecha.astype(float)
        
    return pd.DataFrame(fecha).T

In [ ]:
# Generar DF con datos originales

def _dataBaseOriginal(filename):
    dataBase = pd.read_csv(filename, parse_dates=['FECHA'])
    
    dataBiciMad = dataBase.groupby(['ESTACION','ANIO','MES','DIA','TEMPORADA','DIA_SEMANA', 'Es_Festivo', 'Es_FinSemana',
                          ]).agg(DEMANDA=('DEMANDA', 'sum'),
                                 TEMP_MAX= ('TEMPERATURA','max'),
                                 TEMP_MIN = ('TEMPERATURA','min'),
                                 HUMEDAD = ('HUMEDAD','mean'),
                                 VIENTO = ('VIENTO','mean'),
                                 PRESION = ('PRESION', 'mean'),
                                 PRECIPITACION_1h = ('PRECIPITACION_1h', 'sum'),
                                 PRECIPITACION_3h = ('PRECIPITACION_3h', 'sum'),
                                 DESC_TIEMPO = ('DESC_TIEMPO', lambda x: x.value_counts().index[0]))
    dataBiciMad = dataBiciMad.reset_index()    
    
    return dataBiciMad


In [ ]:
# Generar DF con datos originales pero ESTACION=0 y sin DEMANDA

def _dataBaseOriginalNula(filename):
    dataBase = pd.read_csv(filename, parse_dates=['FECHA'])
    
    dataBiciMad = dataBase.groupby(['ESTACION','ANIO','MES','DIA','TEMPORADA','DIA_SEMANA', 'Es_Festivo', 'Es_FinSemana',
                          ]).agg(
                                 TEMP_MAX= ('TEMPERATURA','max'),
                                 TEMP_MIN = ('TEMPERATURA','min'),
                                 HUMEDAD = ('HUMEDAD','mean'),
                                 VIENTO = ('VIENTO','mean'),
                                 PRESION = ('PRESION', 'mean'),
                                 PRECIPITACION_1h = ('PRECIPITACION_1h', 'sum'),
                                 PRECIPITACION_3h = ('PRECIPITACION_3h', 'sum'),
                                 DESC_TIEMPO = ('DESC_TIEMPO', lambda x: x.value_counts().index[0]))
    dataBiciMad = dataBiciMad.reset_index()    
    dataBiciMad['ESTACION'] = 0
    
    return dataBiciMad

In [ ]:
# %run "../1. Librerias Mongo/MongoDB_Connections.ipynb"
# %run "../1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"

# db_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 
#                           'Meteo_US_NivelHora')
# data_Meteo = _baseMeteorologia(db_Meteo)
# data_festivos = _baseFestivos()
# dataDummy = _dataBaseOriginalNula("../../Data/DataFrame_Final_Cierre_Cluster.csv")

# cat_cols= ['MES', 'DIA_SEMANA', 'TEMPORADA', 'Es_Festivo', 'Es_FinSemana','DESC_TIEMPO']
# num_cols= ['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION']


In [ ]:
# fecha = pd.to_datetime('01/09/2020', format='%d/%m/%Y')
# meteo = data_Meteo[data_Meteo['FECHA']==fecha.strftime('%m/%d/%Y')]

# meteo

In [ ]:
# dataDummy.info()

In [ ]:
# dataDummy

In [ ]:
# fecha = pd.to_datetime('01/09/2020', format='%d/%m/%Y')
# estacion = 1

# FechaAPredecir = _estacionDataFechaNueva(estacion, fecha, data_Meteo, data_festivos, dataDummy, cat_cols, num_cols)

In [ ]:
# FechaAPredecir

In [ ]:
# FechaAPredecir.info()